# Classificação de Documentos

Numa tarefa de classificação, usamos um algoritmo capaz de aprender, a partir de alguns exemplos, como classificar novos elementos. No caso de texto, precisamos oferecer alguns documentos já classificados. No exemplo abaixo usamos notícias do corpus Fake.br já classificadas como verdadeiras ou falsas. O algoritmo usado é o Naive Bayes.

In [1]:
# Importando as bibliotecas
import os
import pandas as pd

## Lendo as notícias dos arquivos
Neste exemplo vamos usar uma amostra do corpus Fake.br (https://github.com/roneysco/Fake.br-Corpus). Precisamos selecionar o diretório contendo os documentos. Neste diretório existem dois outros diretórios, o "fake", no qual estão os arquivos de notícias falsas, e o "true", no qual estão as notícias verdadeiras.

In [2]:
documentos = []

# Lê documentos dos diretórios e monta o dataframe
for category in os.listdir('fake-br-corpus-sample'):
    for doc in os.listdir(f'fake-br-corpus-sample/{category}'):
        name = doc.split('.')[0]
        path = f'fake-br-corpus-sample/{category}/{doc}'
        content = open(path, "r", encoding="utf-8").read()
        documento = {"CATEGORY" : category, "NAME": name, "PATH": path, "CONTENT": content}
        documentos.append(documento)

df = pd.DataFrame(documentos)

## Visualização dos dados
Pode-se visualizar, utilizando o DataFrame criado, as notícias e campos de dados associados a elas.

In [3]:
df.head(10)

,CATEGORY,NAME,PATH,CONTENT
0,true,14,fake-br-corpus-sample/true/14.txt,Raquel Dodge reitera pedido de Janot para resc...
1,true,3,fake-br-corpus-sample/true/3.txt,﻿Após o prefeito de Manaus Arthur Virgílio (PS...
2,true,15,fake-br-corpus-sample/true/15.txt,Anthony Garotinho deixa Bangu com festa de ali...
3,true,2,fake-br-corpus-sample/true/2.txt,Em evento realizado nesta terça-feira para div...
4,true,16,fake-br-corpus-sample/true/16.txt,Conselho de Ética do PMDB decide expulsar a se...
5,true,8,fake-br-corpus-sample/true/8.txt,Cuba comemorará o primeiro aniversário da mort...
6,true,18,fake-br-corpus-sample/true/18.txt,Fachin nega novo pedido de liberdade de Joesle...
7,true,13,fake-br-corpus-sample/true/13.txt,Gilmar nega pedido de Miller para se declarar ...
8,true,20,fake-br-corpus-sample/true/20.txt,PMDB aprova mudança de nome e passa a ser cham...
9,true,9,fake-br-corpus-sample/true/9.txt,s doleiros acusados de lavar dinheiro roubado ...


## Preprocessamento do texto das notícias
Nas análises a seguir, vamos focar na coluna CONTENT, que é a que contém o texto das notícias no dataset. Primeiramente, obtemos as stopwords definidas para o portugês no NLTK. Stopwords são palavras que são muito comuns e por conta disso podem atrapalhar as análises. Nos próximos passos vamos remover estas palavras dos textos.
A função remove_acentos definida abaixo converte caracteres acentuados em caracteres comuns. Vamos utilizá-la para remover estes caracteres especiais.

In [4]:
# Descomente e execute os comandos abaixo se for o primeiro uso dos recursos do NLTK
# nltk.download('stopwords')
# nltk.download('punkt')

In [5]:
import nltk
from nltk.tokenize import word_tokenize
import unicodedata

stopwords = nltk.corpus.stopwords.words('portuguese')

def remove_acentos(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

stopwords = [remove_acentos(palavra) for palavra in stopwords]

def normaliza_texto(txt):
    return ' '.join([word for word in word_tokenize(str.lower(remove_acentos(txt))) if word not in stopwords and word.isalpha()])

df['CONTENT_NORMALIZADO'] = df.apply(lambda linha: normaliza_texto(str(linha['CONTENT'])), axis = 1)

print(df['CONTENT_NORMALIZADO'].head())

0    raquel dodge reitera pedido janot rescindir de...
1    prefeito manaus arthur virgilio psdb ter envia...
2    anthony garotinho deixa bangu festa aliados at...
3    evento realizado nesta divulgar jogo estrelas ...
4    conselho etica pmdb decide expulsar senadora k...
Name: CONTENT_NORMALIZADO, dtype: object


## Amostragem de dados para treinamento
Para separar os nossos documentos entre dados de treinamento e dados que vamos usar posteriormente para avaliação, usamos um método de amostragem de dados da biblioteca Sklearn. Selecionamos 70% dos documentos para usarmos no treinamento classificador. 

In [6]:
from sklearn.model_selection import train_test_split

# divide dados em treinamento e teste
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)

df_test

,CATEGORY,NAME,PATH,CONTENT,CONTENT_NORMALIZADO
19,true,11,fake-br-corpus-sample/true/11.txt,Anthony Garotinho é ouvido mais uma vez sobre ...,anthony garotinho ouvido vez sobre suposta agr...
16,true,17,fake-br-corpus-sample/true/17.txt,﻿Juiz determina soltura de 4 presos por pensão...,determina soltura presos pensao alimenticia co...
15,true,10,fake-br-corpus-sample/true/10.txt,"Bolsonaro é um liberal completo, diz president...",bolsonaro liberal completo diz presidente psl ...
26,fake,18,fake-br-corpus-sample/fake/18.txt,Joesley chorou ao entrar na cela e está sem co...,joesley chorou entrar cela comer tomar banho d...
4,true,16,fake-br-corpus-sample/true/16.txt,Conselho de Ética do PMDB decide expulsar a se...,conselho etica pmdb decide expulsar senadora k...
12,true,4,fake-br-corpus-sample/true/4.txt,﻿Doria vai receber Zé Celso após reunião com r...,vai receber ze celso apos reuniao representant...
37,fake,19,fake-br-corpus-sample/fake/19.txt,Nós vamos estuprar os corruptos afirmam pres...,vamos estuprar afirmam presos papuda mensagens...
27,fake,13,fake-br-corpus-sample/fake/13.txt,Gilmar Mendes age como masoquista ao permitir ...,gilmar mendes age masoquista permitir miller f...
39,fake,11,fake-br-corpus-sample/fake/11.txt,Garotinho ganha prêmio após teatro sobre surr...,garotinho ganha premio apos teatro sobre vai p...
6,true,18,fake-br-corpus-sample/true/18.txt,Fachin nega novo pedido de liberdade de Joesle...,fachin nega novo pedido liberdade joesley bati...


## Contrução do modelo de bag of words para os dados de treinamento e submissão do modelo ao Naive Bayes
Para que o algoritmo de classificação consiga interpretar os documentos, precisamos construir a matriz de documentos X palavras (modelo bag of words). Foi criado um Pipeline para deixar concisa a sequência de passos para preparar os textos para submeter ao Naive Bayes. Inicialmente, foi criado um feature vector contendo a quantidade de vezes que cada palavra aparece no documento. Na sequência, o dicionário de palavras gerado no passo anterior foi submetido ao TF-IDF, para que o peso das palavras seja ajustado de acordo com a importância (rartidade) do termo e sua frequência no documento. Por fim, o dicionário é submetido ao Naive Bayes.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

data_train = df_train['CONTENT_NORMALIZADO']
target_train = df_train['CATEGORY']

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])
text_clf = text_clf.fit(data_train, target_train)

## Classificação dos documentos de teste
Usando o mesmo algoritmo treinado, fazemos a classificação dos documentos separados para testes (com a categoria removida).

In [8]:
data_test = df_test['CONTENT_NORMALIZADO']
target_test = df_test['CATEGORY']

predicted = text_clf.predict(data_test)

Abaixo mostramos as métricas de precision, recall e f-score dos resultados. Como se pode ver, o classificador não apresentou um bom desempenho. Isso era esperado, considerando que este é um dataset muito pequeno para uma boa classificação (exercício: replique os testes no dataset completo). Na sequência exibimos os dados classificados para se inspecionar os erros.

In [9]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(target_test, predicted, average='macro')

(0.4857142857142857, 0.4857142857142857, 0.48571428571428577, None)

In [10]:
pd.DataFrame({'Content': df_test['CONTENT_NORMALIZADO'], 'Predicted': predicted, 'True': target_test})

,Content,Predicted,True
19,anthony garotinho ouvido vez sobre suposta agr...,true,true
16,determina soltura presos pensao alimenticia co...,fake,true
15,bolsonaro liberal completo diz presidente psl ...,fake,true
26,joesley chorou entrar cela comer tomar banho d...,true,fake
4,conselho etica pmdb decide expulsar senadora k...,true,true
12,vai receber ze celso apos reuniao representant...,fake,true
37,vamos estuprar afirmam presos papuda mensagens...,true,fake
27,gilmar mendes age masoquista permitir miller f...,true,fake
39,garotinho ganha premio apos teatro sobre vai p...,fake,fake
6,fachin nega novo pedido liberdade joesley bati...,true,true
